### WebScrapping dos dados

Essa primeira parte consiste na coleta dos dados, com ferramentas de WebScrapping como Selenium e BeautifulSoup, do site de avaliações [IMDb](https://www.imdb.com/title/tt0388629/episodes/?ref_=tt_eps_sm).

As informações são relacionadas às avaliações dos episódios do anime One Piece, que se configura como um dos animes mais populares e com maior quantidade de episódios (ainda contando) da atualidade.

#### Bibliotecas utilizadas

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from bs4 import BeautifulSoup
import requests
import json
import pandas as pd
import datetime

#### Link para cada episódio com Selenium

Essa etapa serve para acessar a página principal de avaliações com todos os episódios e pegar o link de cada um para direcionar à uma página com informações exclusivas de cada episódio. 

Não foi possível obter os dados nessa página principal, por isso foi necessário coletar o link externo para cada episódio.

In [ ]:
# Inicie o navegador (você precisa ter o driver correspondente, ex: chromedriver)
driver = webdriver.Chrome()

# Acesse a página
url = 'https://www.imdb.com/title/tt0388629/episodes/?ref_=tt_eps_sm'

driver.get(url)

# Aguarde alguns segundos para a página carregar
time.sleep(3)

# Localize e clique no botão "Mostrar tudo" (ajuste o seletor conforme necessário)
# Aguarde até que o botão seja clicável
try:
    button1 = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="__next"]/main/div/section/div/section/div/div[1]/section[2]/section[2]/article[51]/div/span[2]')))

    # Rolar até o botão antes de clicar
    driver.execute_script("arguments[0].scrollIntoView(true);", button1)
    time.sleep(3)  # Aguarde um segundo para garantir que o botão está visível

    button1.click()  # Clica no botão "Tudo"

    #time.sleep(45)
    time.sleep(10)
    
    episodes = list()

    for i in range(1,1118):

        try:
            episode = driver.find_element(By.XPATH, f'//*[@id="__next"]/main/div/section/div/section/div/div[1]/section[2]/section[2]/article[{i}]/div/div/div[3]/div[1]/h4/div/a').get_attribute('href')
            
        except:
            try:
                episode = driver.find_element(By.XPATH, f'//*[@id="__next"]/main/div/section/div/section/div/div[1]/section[2]/section[2]/article[{i}]/div/div/div[3]/div[2]/h4/div/a').get_attribute('href')
          
            except:
                print(f'Algo deu errado com o episódio: {i}')

            else:
                # Segunda tentativa foi bem-sucedida
                print(f'Episódio {i} encontrado: {episode}')
        else:
            # Primeira tentativa foi bem-sucedida
            print(f'Episódio {i} encontrado: {episode}')

        episodes.append(episode)            

except Exception as e:
    print(e)

finally: 
    # Feche o navegador
    driver.close()

#### Coleta dos dados de cada episódio com Beutiful Soup

Essa etapa consistiu em utilizar o BeautifulSoup para obter as informações de cada link entre os 1117 episódios e gerar um DataFrame com todos os dados.

In [ ]:
episodes_dict = {
    'Episódio': [], # content_data['series']['episodeNumber']['episodeNumber']
    'Título': [], #content_data['titleText']['text']
    'Lançamento': [],
    'Votos': [], #content_data['ratingsSummary']['voteCount']
    'Avaliação': []     #content_data['ratingsSummary']['aggregateRating']
    }

ep_count = 1

for episode in episodes:

    # Simule o comportamento de um navegador adicionando um cabeçalho 'User-Agent'
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    # Faça a requisição usando o cabeçalho
    response = requests.get(episode, headers=headers)

    soup = BeautifulSoup(response.content, 'html.parser')  

    script = soup.find('script', id="__NEXT_DATA__", type="application/json")

    json_data = script.string
                
    # Converta a string JSON em um dicionário Python
    data = json.loads(json_data)

    # Exemplo: Acesse as chaves dentro do dicionário
    content_data = data['props']['pageProps']['aboveTheFoldData']

    episodes_dict['Episódio'].append(content_data['series']['episodeNumber']['episodeNumber'])
    episodes_dict['Título'].append(content_data['titleText']['text'])

    dia, mês, ano = content_data['releaseDate']['day'], content_data['releaseDate']['month'], content_data['releaseDate']['year']
    episodes_dict['Lançamento'].append(datetime.date(ano, mês, dia))
    
    episodes_dict['Qtd de votos'].append(content_data['ratingsSummary']['voteCount'])
    episodes_dict['Avaliação média'].append(content_data['ratingsSummary']['aggregateRating'])

    print(f'Episódio {ep_count} processado.')

    ep_count += 1

In [ ]:
df_episodes = pd.DataFrame(episodes_dict)
df_episodes

In [24]:
df_episodes.to_excel('.../episodes.xlsx', index=0)